## https://qiita.com/paulxll/items/98cd3d3d8adbf6197660

In [65]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import LongType,  DoubleType, IntegerType, StringType, StructType, StructField
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [66]:
df = spark.createDataFrame(
    [
        ("1", 1.0), ("1", 2.0),
        ("1", 5.0), ("1", 7.0),
        ("2", 3.0), ("2", 5.0),
        ("2", 10.0), ("2", 0.0),
        ("3", 6.0), ("3", 6.0),
        ("3", 7.0), ("3", 8.0),
        ("4", 5.0), ("4", 9.0),
        ("4", 2.0), ("4", 2.0),
        ("5", 7.0), ("5", 1.0),
        ("5", 2.0), ("5", 4.0)
        ],
    ("id", "v"))

In [67]:
df.show()

+---+----+
| id|   v|
+---+----+
|  1| 1.0|
|  1| 2.0|
|  1| 5.0|
|  1| 7.0|
|  2| 3.0|
|  2| 5.0|
|  2|10.0|
|  2| 0.0|
|  3| 6.0|
|  3| 6.0|
|  3| 7.0|
|  3| 8.0|
|  4| 5.0|
|  4| 9.0|
|  4| 2.0|
|  4| 2.0|
|  5| 7.0|
|  5| 1.0|
|  5| 2.0|
|  5| 4.0|
+---+----+



### scaler

In [68]:
#pandas_udfに、入力値の型(double)と、PandasUDFType(SCALAR)を指定
@pandas_udf('double', PandasUDFType.SCALAR)
def pandas_plus_one(v):
    """
    1を足して返すだけの関数
    """
    return v + 1

df = df.withColumn('v2', pandas_plus_one(df.v))
df.show()

+---+----+----+
| id|   v|  v2|
+---+----+----+
|  1| 1.0| 2.0|
|  1| 2.0| 3.0|
|  1| 5.0| 6.0|
|  1| 7.0| 8.0|
|  2| 3.0| 4.0|
|  2| 5.0| 6.0|
|  2|10.0|11.0|
|  2| 0.0| 1.0|
|  3| 6.0| 7.0|
|  3| 6.0| 7.0|
|  3| 7.0| 8.0|
|  3| 8.0| 9.0|
|  4| 5.0| 6.0|
|  4| 9.0|10.0|
|  4| 2.0| 3.0|
|  4| 2.0| 3.0|
|  5| 7.0| 8.0|
|  5| 1.0| 2.0|
|  5| 2.0| 3.0|
|  5| 4.0| 5.0|
+---+----+----+



In [69]:
@pandas_udf('double', PandasUDFType.SCALAR)
def pandas_mean_diff(pds):
    """
    平均値との差分を返す関数
    """
    return pds - pds.mean()

In [70]:
df = df.withColumn('mean_diff', pandas_mean_diff(col("v")))
df.show(10, False)

+---+----+----+---------+
|id |v   |v2  |mean_diff|
+---+----+----+---------+
|1  |1.0 |2.0 |-0.5     |
|1  |2.0 |3.0 |0.5      |
|1  |5.0 |6.0 |-1.0     |
|1  |7.0 |8.0 |1.0      |
|2  |3.0 |4.0 |-1.0     |
|2  |5.0 |6.0 |1.0      |
|2  |10.0|11.0|4.5      |
|2  |0.0 |1.0 |-5.5     |
|3  |6.0 |7.0 |0.5      |
|3  |6.0 |7.0 |0.5      |
+---+----+----+---------+
only showing top 10 rows



### GROUPED_AGG

In [71]:
@pandas_udf("double", PandasUDFType.GROUPED_AGG)
def filtered_mean(pds1, pds2):
    """
    カラム:vとカラム:v2の差分の平均をとる関数
    """
    pdf = pds1 - pds2
    return pdf.mean()

In [72]:
df.groupBy("id").agg(filtered_mean("v", "v2")).show(10, False)

+---+--------------------+
|id |filtered_mean(v, v2)|
+---+--------------------+
|3  |-1.0                |
|5  |-1.0                |
|1  |-1.0                |
|4  |-1.0                |
|2  |-1.0                |
+---+--------------------+



### GROUPED_MAP

In [73]:
schema = StructType([
  StructField("id", StringType(), False),
  StructField("v", DoubleType(), False),
  StructField("v2", DoubleType(), False),
  StructField("cluster", IntegerType(), False)
])

In [74]:
df = df.select("id","v","v2")
df.show()

+---+----+----+
| id|   v|  v2|
+---+----+----+
|  1| 1.0| 2.0|
|  1| 2.0| 3.0|
|  1| 5.0| 6.0|
|  1| 7.0| 8.0|
|  2| 3.0| 4.0|
|  2| 5.0| 6.0|
|  2|10.0|11.0|
|  2| 0.0| 1.0|
|  3| 6.0| 7.0|
|  3| 6.0| 7.0|
|  3| 7.0| 8.0|
|  3| 8.0| 9.0|
|  4| 5.0| 6.0|
|  4| 9.0|10.0|
|  4| 2.0| 3.0|
|  4| 2.0| 3.0|
|  5| 7.0| 8.0|
|  5| 1.0| 2.0|
|  5| 2.0| 3.0|
|  5| 4.0| 5.0|
+---+----+----+



In [76]:
@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def grouped_km(pdf):
    """
    groupごとにKMeansによるクラスタリングを実施しデータを5つに分ける。
    """
    from sklearn.cluster import KMeans
    km = KMeans(n_clusters=2, random_state=71)
    pdf.loc[:, "cluster"] = km.fit_predict(pdf.loc[:, ["v", "v2"]])
    return pdf

In [78]:
result = df.groupBy("id").apply(grouped_km)
result.show()

+---+----+----+-------+
| id|   v|  v2|cluster|
+---+----+----+-------+
|  3| 6.0| 7.0|      1|
|  3| 6.0| 7.0|      1|
|  3| 7.0| 8.0|      0|
|  3| 8.0| 9.0|      0|
|  5| 7.0| 8.0|      1|
|  5| 1.0| 2.0|      0|
|  5| 2.0| 3.0|      0|
|  5| 4.0| 5.0|      0|
|  1| 1.0| 2.0|      1|
|  1| 2.0| 3.0|      1|
|  1| 5.0| 6.0|      0|
|  1| 7.0| 8.0|      0|
|  4| 5.0| 6.0|      0|
|  4| 9.0|10.0|      1|
|  4| 2.0| 3.0|      0|
|  4| 2.0| 3.0|      0|
|  2| 3.0| 4.0|      0|
|  2| 5.0| 6.0|      0|
|  2|10.0|11.0|      1|
|  2| 0.0| 1.0|      0|
+---+----+----+-------+

